# 客户价值分析

要实现精细化运营，就需要对用户进行分层，筛选出各层次的用户，这样运营才能针对性的制作策略去运营。我今天就来说下用户分层经典的模型**RFM**模型与**聚类分析**模型

## 一、什么是RFM模型

RFM是3个指标的缩写，最近一次消费时间间隔（Recency），消费频率（Frequency），消费金额（Monetary）。而RFM模型就是通过这三项指标，来描述客户的价值状况，从而得到分群的客户。其中

* R是指用户的最近一次消费时间距现在有多长时间了，这个指标反映用户流失与复购（粘性）。

* F是指用户在指定观察的周期内消费了几次。这个指标反映了用户的消费活跃度（忠诚度）。

* M是指用户在指定的观察周期内在平台花了多少钱，这个指标反映用户对公司贡献的价值（营收）。

## 二、什么时候用

任何模型最终都是腰围业务服务的，并不任何时候、任何数据都可以套用这个模型最客户分群的。用RFM模型时需要考虑下面几个因素

* 需要客户有一定的消费频次，像房子、耐用品、家私这类，客户不会隔三差五去消费这些东西，所以这类的消费数据要做客户价值分析的话，RFM不是一个好的选择

* 需要客户有消费金额，对企业产生价值

* 需要记录有用户ID，这样RFM分层完后才能对应上

* RFM 更使用于传统企业里的运营商、银行、航空，以及互联网消费行业

## 三、怎么用

下面我就接着上一篇文章中京东消费者数据，用python来实现RFM模型，由于数据集中缺少购买金额这个数据，所以这个维度我就先忽略不看，重要的还是思路

In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [182]:
jd_consumer=pd.read_excel('./京东消费者分析数据.xlsx',sheet_name='JD_Fnl')

In [183]:
jd_consumer.head()

,customer_id,product_id,action_date,action_id,type,age_range,gender,customer_register_date,customer_level,city_level,brand,shop_id,category,product_market_date,vender_id,fans_number,vip_number,shop_register_date,shop_category,shop_score
0,976695,328360,2018-03-05 08:52:00,10633744,PageView,5.0,W,2017-06-17,6,3.0,LukFook,2959,Bracelet,2017-10-05,4322,1159,36596,2017-07-04,Jewellery Accessories,9.043610
1,395399,141632,2018-03-20 10:55:00,5472028,PageView,4.0,M,2018-01-21,6,5.0,illuma,4203,Milk Power,2015-06-12,3563,2411,65893,2013-08-08,Mother and Infant,9.537151
2,1068082,5924,2018-04-01 13:56:00,3853455,PageView,6.0,W,2012-08-13,1,4.0,Li-Ning,8002,Badminton,2013-06-14,1457,213610,357580,2013-05-23,Outdoor Sports,9.571139
3,1068082,10723,2018-04-01 13:45:00,3853455,PageView,6.0,W,2012-08-13,1,4.0,Li-Ning,8002,Badminton,2011-07-19,1457,213610,357580,2013-05-23,Outdoor Sports,9.571139
4,735392,4559,2018-02-06 20:25:00,9434940,PageView,6.0,M,2009-09-15,7,5.0,Other,6944,Cookie,2015-11-16,355,63969,85597,2014-10-31,Food,9.665680


* 观察购买次数频率的分布

In [184]:
jd_consumer[jd_consumer['type']=='Order'].groupby('customer_id').size().sort_values(ascending=False)

customer_id
653834     3
813206     2
228162     2
507444     2
1496979    2
          ..
1087899    1
1087819    1
1087596    1
1087560    1
64         1
Length: 10652, dtype: int64

只选择消费食品的用户用来分析，因为用户在一定时间内购买食品的频次比其他电子产品、服饰等更活跃

* 对数据进行整合，以2018年04月15号到2018年3月1号为观察的时间区间

In [185]:
jd_consumer_food.head()

,customer_id,action_date,type
13080,659379,2018-02-01,Order
13089,292645,2018-02-01,Order
13090,28082,2018-02-01,Order
13092,597722,2018-02-01,Order
13095,728665,2018-02-01,Order


In [186]:
#对列重命名
jd_consumer.rename(columns={'action_date':'action_time'},inplace=True)
#提取时间中的小时时段
jd_consumer['action_date']=jd_consumer['action_time'].apply(lambda x:pd.datetime.strftime(x,'%Y-%m-%d'))
jd_consumer['action_month']=jd_consumer['action_time'].apply(lambda x:pd.datetime.strftime(x,'%Y-%m'))
jd_consumer_food=jd_consumer[(jd_consumer['type']=='Order') & 
                             (jd_consumer['shop_category']=='Food') & 
                            (jd_consumer['action_month']=='2018-03')][['customer_id',
                                                                      'action_date','type']].drop_duplicates()

* R

In [187]:
idx=jd_consumer_food.groupby('customer_id').size().sort_values(ascending=False)[:1000].index

In [188]:
jd_rfm=jd_consumer_food[jd_consumer_food['customer_id'].isin(idx)].groupby('customer_id').agg({'action_date':'max','type':'count'})

In [189]:
jd_rfm['R']=(pd.to_datetime('2018-04-15')-pd.to_datetime(jd_rfm['action_date'])).dt.days
jd_rfm.rename(columns={'type':'F'},inplace=True)

In [190]:
jd_rfm=jd_rfm.drop('action_date',axis=1)

In [191]:
jd_consumer_food.head()

,customer_id,action_date,type
13512,1086601,2018-03-01,Order
13514,182726,2018-03-01,Order
13518,1393060,2018-03-01,Order
13520,1458445,2018-03-01,Order
13527,980336,2018-03-01,Order


In [198]:
rmd = jd_rfm['R'].mean()
fmd = jd_rfm['F'].mean()
rmd,fmd

(29.321, 1.004)

In [203]:
def customer_type(frame): 
    customer_type = []
    for i in range(len(frame)):
        if frame.iloc[i,1]<=rmd and frame.iloc[i,0]>=fmd :
            customer_type.append('重要价值用户')
        elif  frame.iloc[i,1]>rmd and frame.iloc[i,0]>=fmd :
            customer_type.append('重要唤回用户')
        elif  frame.iloc[i,1]<=rmd and frame.iloc[i,0]<fmd :
            customer_type.append('重要深耕用户')
        elif  frame.iloc[i,1]>rmd and frame.iloc[i,0]<fmd :
            customer_type.append('即将流失用户')
    frame['classification'] = customer_type

In [206]:
customer_type(jd_rfm)

print('不同类型的客户总数：')
print('--------------------')
jd_rfm.groupby(by='classification').size().reset_index(name='客户数')

不同类型的客户总数：
--------------------


,classification,客户数
0,即将流失用户,499
1,重要价值用户,3
2,重要唤回用户,1
3,重要深耕用户,497


可以看出流失用户、重要深耕用户占了非常大的比重

运营可以通过策划优惠活动、发优惠券等活动来促进重要深耕用户转化到重要价值用户

对于即将流失可以进行推送激活，触达到流失用户，增加回流的概率


## 四、用户分类完整表

由于此数据缺失金额，所以没能进行一个完整的RFM分析，不过具体的思路上面已经阐释清楚了，以下附上一个完整的RFM用户分类表作为实战参考

| 用户类型     | 最近一次消费时间间隔（Recency） | 消费频率（Frequency） | 消费金额（Monetary） | 运营手段                                       |
| ------------ | ------------------------------- | --------------------- | -------------------- | ---------------------------------------------- |
| 重要价值用户 | 高                              | 高                    | 高                   | 最优质用户，应提高满意度，增加留存             |
| 重要唤回用户 | 低                              | 高                    | 高                   | 购买频率低，可通过活动等提高购买率             |
| 重要深耕用户 | 高                              | 低                    | 高                   | 推送增加触达，以免流失                         |
| 重要挽留用户 | 低                              | 低                    | 高                   | 潜在有价值的用户，了解原因，进行挽留           |
| 潜力用户     | 高                              | 高                    | 低                   | 忠诚用户，但累计消费金额低，可引导进行消费升级 |
| 新用户       | 低                              | 高                    | 低                   | 新用户，可利用低门槛优惠券等方式吸引           |
| 一般维持用户 | 高                              | 低                    | 低                   | 需分析进行运营激活                             |
| 流失用户     | 低                              | 低                    | 低                   | 相当于流失用户，推送重新激活                   |



## 五、后续

RFM可以根据用户的这三类数据进行分层，但实际中很多其它维度的数据，所以为了更好的实现精细化运营，需要更多的维度参与进来。